In [9]:
import os
import csv
import pandas as pd
import numpy as np

## Analysis on HMDA file only

- todo - this is only the mini. Once prep code good... update to all (and some steps below will be done in the prep file already)
- todo - write up findings after this is run on all


In [14]:
hmda = pd.read_csv('../input_data_clean/state_AZ-CA_clean_MINI.csv')

# Our tables will contract BMO and BOW to all other banks
def lei_map(lei):
    if lei == 'QOT5WN9RBKQTFRVKEV31':
        return 'Bank of West'
    elif lei == '3Y4U8VZURTYWI1W2K376':
        return 'BMO Harris'
    elif lei == np.nan:
        return np.nan
    else: 
        return 'All Other Banks'

hmda['which_bank'] = hmda['lei'].map(lei_map)

hmda['approved'] = (hmda['action_taken'] == 2) | (hmda['action_taken'] == 1) 

hmda['application'] = (hmda['action_taken'] < 4 ) 


## Denial rates by product and purpose (aggregate)

In [13]:
# the denial rate = denials/apps
# so restrict to applications

apps = hmda.query('action_taken < 4').copy()

apps['denial'] = apps['action_taken'] == 3

# Create the pivot table
pivot = apps.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')

# add columns for differentials (easier to see)
pivot['diff_BMO'] = pivot['BMO Harris'] - pivot['All Other Banks']
pivot['diff_BOW'] = pivot['Bank of West'] - pivot['All Other Banks']
pivot.style.format('{:,.2f}')

## Avg requested loan size by product and purpose for all applications (aggregate)

In [17]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Avg loan size by product and purpose for approved loans (aggregate)

In [18]:
hmda.query('approved == 1').pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Rate spread by product and purpose (aggregate)

## Median income of applicants by product and purpose (aggregate)

## Median income of approved loans by product and purpose (aggregate)

## Code that produces tables that need headers